PROCESS DOCUMENT

In [48]:
import os
from IPython.display import Markdown
from utils.document_processor import DocumentProcessor
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

# Inisialisasi pemrosesan dokumen
docs = DocumentProcessor()
DATA_PATH = "./data"
extracted_docs = []
valid_extensions = ('.pdf', '.docx', '.txt')

# **Baca dan Proses Semua Dokumen**
all_texts = []
metadata_list = []
file_texts = {}

# Baca dan proses dokumen
for filename in os.listdir(DATA_PATH):
    if not filename.lower().endswith(valid_extensions):
        continue  # Skip file yang tidak didukung

    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, "rb") as f:
        document = f.read()
        result = docs.process_document(document, filename)
        content, images, tables, plain_text = result

        # **Simpan teks per file**
        file_texts[filename] = plain_text
        
        # **Split teks menjadi chunk**
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
        chunks = text_splitter.split_text(plain_text)

        # **Tambahkan metadata (nama file sebagai sumber)**
        for chunk in chunks:
            all_texts.append(chunk)
            metadata_list.append({"source": filename})

# content, images, tables, plain_text = extracted_docs[0]  # Ambil file pertama

# Gunakan Hugging Face embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = FAISS.from_texts(all_texts, embedding_model, metadatas=metadata_list)

retriever = vectorstore.as_retriever()

C:\Users\62812\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (103.42, 438.678, 526.0919999999999, 621.9063333333334)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
C:\Users\62812\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (139.42, 638.148, 481.82, 816.2009963636362)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
C:\Users\62812\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (103.42, 692.028, 523.304, 829.176)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
C:\Users

TEST OLLAMA MODEL

In [49]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

In [50]:
OLLAMA_MODEL = "llama3.2"
COLLECTION_NAME = "ollama_vectore_test"

In [51]:
template = """
  You are a helpful assistant for text summarization. 
  Only include information that is part of the document. 
  Do not include your own opinion or analysis.

  Document: 
  "{document}"
  Summary:
"""

prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model=OLLAMA_MODEL)


RAG AGENTS

In [52]:
# Retrieval QA Chain
retrieval_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

# Fungsi untuk Menampilkan Jawaban dengan Sumber
def retrieve_answer(query):
    response = retrieval_chain.invoke({"query": query})
    answer = response["result"]
    sources = {doc.metadata["source"] for doc in response["source_documents"]}
    
    source_text = "\n\n**Sumber:**\n" + "\n".join(sources)
    return answer + source_text

# Fungsi untuk Ringkasan per Dokumen
def summarize_document(doc_name):
    if doc_name in file_texts:
        summary_prompt = f"Ringkas dokumen berikut:\n\n{file_texts[doc_name]}\n\nRingkasan:"
        summary = model.invoke(summary_prompt)
        return f"**Ringkasan dari {doc_name}:**\n{summary}"
    else:
        return f"Dokumen '{doc_name}' tidak ditemukan dalam sistem."
    
# Fungsi untuk Ringkasan Semua Dokumen
def summarize_all_documents(_=None):  # Tambahkan parameter dengan default None
    summaries = []
    for filename, text in file_texts.items():
        summary_prompt = f"Ringkas dokumen berikut:\n\n{text}\n\nRingkasan:"
        summary = model.invoke(summary_prompt)
        summaries.append(f"**Ringkasan dari {filename}:**\n{summary}")
    
    return "\n\n".join(summaries)

# Definisikan Tools untuk Agent
tools = [
    Tool(
        name="retrieval_qa",
        func=retrieve_answer,
        description="Gunakan ini untuk menjawab pertanyaan berdasarkan dokumen yang tersedia."
    ),
    Tool(
        name="summarize_document",
        func=summarize_document,
        description="Gunakan ini untuk meringkas satu dokumen. Masukkan nama file dokumen sebagai input."
    ),
    Tool(
        name="summarize_all_documents",
        func=summarize_all_documents,
        description="Gunakan ini untuk meringkas semua dokumen yang tersedia."
    )
]

In [53]:
# Inisialisasi Agent dengan Multi-Query Handling
agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# Coba Jalankan Agent dengan Pertanyaan
queries = [
    "Apa faktor utama yang mempengaruhi kesehatan mental?",
    "Ringkas dokumen 'Kesehatan_Mental_Sumber_Daya_Manusia_Ind.pdf'.",
    "Ringkas semua dokumen tentang kesehatan mental."
]

# Jalankan dan Tampilkan Hasil
for query in queries:
    print(f"\n### Pertanyaan: {query}\n")
    response = agent.invoke({"input": query})
    Markdown(response["output"])


### Pertanyaan: Apa faktor utama yang mempengaruhi kesehatan mental?



> Entering new AgentExecutor chain...
Question: Apa faktor utama yang mempengaruhi kesehatan mental?

Action: retrieval_qa
Action Input: "faktor utama yang mempengaruhi kesehatan mental"
Observation: Saya tidak mengetahui jawaban yang pasti untuk pertanyaan tersebut berdasarkan konteks yang diberikan.

**Sumber:**
962-Article Text-3898-1-10-20210930.pdf
Kesehatan_Mental_Sumber_Daya_Manusia_Ind.pdf
Kesehatan_Mental_Masyarakat_Indonesia_Pe.pdf
Thought:Question: Apa faktor utama yang mempengaruhi kesehatan mental?
Thought: Faktor-faktor yang mempengaruhi kesehatan mental sangat kompleks dan beragam. Berdasarkan sumber-sumber yang tersedia, beberapa faktor utama yang mempengaruhi kesehatan mental adalah stres, kekurangan tidur, pola makan tidak seimbang, kurangnya aktivitas fisik, riwayat trauma, dan hubungan dengan orang lain.
Action: summarize_document
Action Input: "Kesehatan_Mental_Sumber_Daya_Manusia_Ind.pdf"
Obs

TEST OPENAI MODEL

In [54]:
from dotenv import load_dotenv
import os
import openai
load_dotenv()

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")

model = "gpt-35-turbo"
response = openai.ChatCompletion.create(
  engine=model,
  messages=[
    {
      "role": "system", 
      "content": """
        You are a helpful assistant for text summarization.
        Only include information that is part of the document. 
        Do not include your own opinion or analysis.
      """
    },
    {
      "role": "user",
      "content": plain_text
    }
  ],
)

Markdown(response.choices[0].message.content)

InvalidRequestError: This model's maximum context length is 8192 tokens. However, your messages resulted in 18397 tokens. Please reduce the length of the messages.